<a href="https://colab.research.google.com/github/senushidinara/LearnScape-AI/blob/main/Wolfram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from IPython.display import Image, display
import json

# ---------------------------------------------
# CONFIG — paste your Wolfram AppID
# ---------------------------------------------
API_KEY = "H34WX632E2"

# ---------------------------------------------
# MAIN FUNCTION — full processing pipeline
# ---------------------------------------------
def wolfram_all(query):
    print("🔍 Querying Wolfram Alpha...")

    # Build URL
    url = (
        f"https://api.wolframalpha.com/v2/query"
        f"?input={query}"
        f"&appid={API_KEY}"
        f"&output=json"
    )

    # Request JSON
    data = requests.get(url).json()

    # Prepare result container
    output = {
        "query": query,
        "pods": [],
        "images": [],
        "plots": [],
        "steps": [],
        "text_blocks": []
    }

    # Extract pods
    pods = data["queryresult"].get("pods", [])

    for pod in pods:
        title = pod["title"]
        subpod = pod["subpods"][0]

        # Save text content
        plaintext = subpod.get("plaintext", "")
        if plaintext:
            output["text_blocks"].append({"title": title, "text": plaintext})

        # Save images
        if "img" in subpod:
            img_url = subpod["img"]["src"]
            output["images"].append({"title": title, "url": img_url})

            # If it's a plot, store separately
            if "Plot" in title or "plot" in title.lower():
                output["plots"].append(img_url)

        # Step-by-step solutions
        if "Step-by-step solution" in title:
            output["steps"].append(plaintext)

        # Store raw pod
        output["pods"].append({
            "title": title,
            "plaintext": plaintext,
            "image": subpod.get("img", {}).get("src", None)
        })

    # Display images directly in Colab
    print("\n📊 Showing visual outputs:\n")
    for img in output["images"]:
        print("Image:", img["title"])
        display(Image(url=img["url"]))

    print("\n✅ Finished.")
    return output


# ---------------------------------------------
# RUN EVERYTHING WITH ONE COMMAND
# ---------------------------------------------
query = "definite integral of x^2 + 2x from 0 to 5"
result = wolfram_all(query)

# Show the cleaned LearnScape-ready dictionary
print("\n============= LEARNSCAPE OUTPUT =============\n")
print(json.dumps(result, indent=2))

🔍 Querying Wolfram Alpha...

📊 Showing visual outputs:

Image: Definite integral


Image: Visual representation of the integral


Image: Riemann sums


Image: Indefinite integral



✅ Finished.

============= LEARNSCAPE OUTPUT =============

{
  "query": "definite integral of x^2 + 2x from 0 to 5",
  "pods": [
    {
      "title": "Definite integral",
      "plaintext": "integral_0^5 x^2 2 x dx = 625/2 = 312.5",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq5ydkim5v.gif"
    },
    {
      "title": "Visual representation of the integral",
      "plaintext": "",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq661p7oc2.gif"
    },
    {
      "title": "Riemann sums",
      "plaintext": "left sum | (625 (n - 1)^2)/(2 n^2) = 625/2 - 625/n + O((1/n)^2)\n(assuming subintervals of equal length)",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq61xvg6cp.gif"
    },
    {
      "title": "Indefinite integral",
      "plaintext": "integral x^2 2 x dx = x^4/2 + constant",
      "image": "https://public5c.wolframalpha.com/files/GIF_xq5zwgenc2.gif"
    }
  ],
  "images": [
    {
      "title": "Definite integral",
      "url": "https

In [2]:
import requests
import json

# YOUR APP ID
APP_ID = "H34WX632E2"

def wolfram_query(query):
    """Send query to Wolfram Alpha Full Results API"""
    url = "https://api.wolframalpha.com/v2/query"

    params = {
        "input": query,
        "appid": APP_ID,
        "output": "JSON",         # important: return JSON not XML
        "format": "image,plaintext"
    }

    response = requests.get(url, params=params)
    data = response.json()

    return data


def parse_wolfram_result(data):
    """Extract clean results from Wolfram JSON"""

    pods = data.get("queryresult", {}).get("pods", [])
    result = {
        "input_interpretation": None,
        "plaintext_results": [],
        "images": [],
        "3d_data": None
    }

    for pod in pods:
        title = pod.get("title", "").lower()

        # Text results / formulas / steps
        if pod.get("subpods"):
            for sp in pod["subpods"]:
                if "plaintext" in sp and sp["plaintext"]:
                    result["plaintext_results"].append({
                        "title": pod.get("title"),
                        "value": sp["plaintext"]
                    })

                # Image results (plots)
                if "img" in sp and "src" in sp["img"]:
                    result["images"].append(sp["img"]["src"])

        # Try extracting 3D data if available
        if "3d" in title or "plot" in title:
            try:
                # Wolfram sometimes puts JSON 3D data inside "minput" or "moutput"
                result["3d_data"] = pod
            except:
                pass

    return result


def pretty_print(result):
    print("==== TEXT RESULTS ====\n")
    for item in result["plaintext_results"]:
        print(f"{item['title']}: {item['value']}\n")

    print("==== IMAGES ====\n")
    for img in result["images"]:
        print(img)

    print("\n==== 3D DATA (raw) ====\n")
    print(json.dumps(result["3d_data"], indent=2))


# -----------------------------
#        RUN EVERYTHING
# -----------------------------
query = "plot x^2 + 2x from -5 to 5 in 3D"
data = wolfram_query(query)
result = parse_wolfram_result(data)
pretty_print(result)

==== TEXT RESULTS ====

Input interpretation: 3D plot | x^2 + 2 x | x = -5 to 5

==== IMAGES ====

https://public6.wolframalpha.com/files/GIF_xtj90b8pgn.gif
https://public6.wolframalpha.com/files/GIF_xtjnqya4pw.gif

==== 3D DATA (raw) ====

null
